In [8]:
import pandas as pd
import requests
import time
import datetime
from datetime import datetime as dt
from tqdm.notebook import tqdm

# bithumb api 호출

In [19]:
url = 'https://api.bithumb.com/public/ticker/all'
params = '{order_currency}_{payment_currency}/{chart_intervals}'
resp = requests.get(url)
coin_list = list(resp.json()['data'].keys())[:-1]
df_list = list(map(lambda i: 'df_' + i, coin_list))
df_coin_dict = {}

In [33]:
to_dt = lambda i: dt.fromtimestamp(i/1000)
def get_dist(df):
    return abs(df['ma90']- df['ma45'])
def get_voldist(df):
    return abs(df['volMA90']- df['volMA45'])

In [34]:
for coin in tqdm(coin_list):
    # crawling part
    chart_intervals = ''
    payment_currency = 'KRW'
    order_currency = coin
    url = f'https://api.bithumb.com/public/candlestick/{order_currency}_{payment_currency}/{chart_intervals}'
    resp = requests.get(url)
    json_coin = resp.json()['data']
    # json to df
    df_coin = pd.DataFrame.from_dict(json_coin)
    df_coin.columns = ['datetime','open','close','high','low','volume']
    df_coin['datetime'] = df_coin['datetime'].apply(to_dt)
    df_coin.set_index('datetime', inplace=True)
    df_coin['open'] = df_coin['open'].astype('float')
    df_coin['close'] = df_coin['close'].astype('float')
    df_coin['high'] = df_coin['high'].astype('float')
    df_coin['low'] = df_coin['low'].astype('float')
    df_coin['volume'] = df_coin['volume'].astype('float')
    # 주가 변동폭 계산
    df_coin['diff'] = df_coin['high'] - df_coin['low']    
    # 이평 이격도 계산
    df_coin['ma90'] = df_coin['close'].rolling(window=90, min_periods=90).mean()
    df_coin['ma45'] = df_coin['close'].rolling(window=45, min_periods=45).mean()
    df_coin['dist'] = df_coin.apply(get_dist, axis=1)
    # 거래량 비율 계산
    df_coin['volMA90'] = df_coin['volume'].rolling(window=90, min_periods=90).mean()
    df_coin['volMA45'] = df_coin['volume'].rolling(window=45, min_periods=45).mean()
    df_coin['voldist'] = df_coin.apply(get_voldist, axis=1)
    df_coin['volRatio'] = abs(1 - df_coin['volume'] / df_coin['volMA90'])
    # 볼린저 밴드 계산
    df_coin['mbb'] = df_coin['close'].rolling(20).mean()
    df_coin['ubb'] = df_coin['mbb'] + 2 * df_coin['close'].rolling(20).std()
    df_coin['lbb'] = df_coin['mbb'] - 2 * df_coin['close'].rolling(20).std()
    df_coin['bolingerdiff'] = df_coin['ubb'] - df_coin['lbb']
    df_coin_dict[coin] = df_coin
# 시간 시 종 고 저 거래량

In [35]:
df_BTC = df_coin_dict['BTC']

# 주가 변동폭

In [ ]:
df_BTC['diff'] = df_BTC['high'] - df_BTC['low']    

# macd(deprecated)

In [12]:
# def get_macd(df):
#     df['ma12'] = df.close.ewm(span=12).mean() # 단기(12) EMA(지수이동평균) 
#     df['ma26'] = df.close.ewm(span=26).mean() # 장기(26) EMA 
#     df['macd'] = df['ma12'] - df['ma26'] # MACD 
#     df['sig'] = df['macd'].ewm(span=9).mean() # Signal 
#     df['osc'] = df['macd'] - df['sig'] # Oscillator

In [13]:
# get_macd(df_BTC)

# 볼린저 밴드

In [ ]:
df_BTC['mbb'] = df_BTC['close'].rolling(20).mean()
df_BTC['ubb'] = mbb + 2 * df_BTC['close'].rolling(20).std()
df_BTC['lbb'] = mbb - 2 * df_BTC['close'].rolling(20).std()
df_BTC['bolingerdiff'] = df_BTC['ubb'] - df_BTC['lbb']

# 거래량 비율

In [31]:
df_BTC['volMA90'] = df_BTC['volume'].rolling(window=90, min_periods=90).mean()
df_BTC['volRatio'] = df_BTC['volume'] / df_BTC['volMA90']

# 상관계수

In [36]:
df_BTC.corr()['diff'].sort_values(ascending=False)

diff            1.000000
bolingerdiff    0.771605
ubb             0.701611
dist            0.696454
high            0.695652
open            0.669920
close           0.665585
mbb             0.649651
low             0.635448
ma45            0.612619
lbb             0.557029
ma90            0.530454
volMA90         0.516306
volMA45         0.505438
volume          0.490017
voldist         0.318133
volRatio        0.200764
Name: diff, dtype: float64

# 회귀분석

In [39]:
import statsmodels.formula.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std

In [43]:
result = sm.ols(formula= 'diff ~ bolingerdiff + dist + volume', data= df_BTC).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   diff   R-squared:                       0.673
Model:                            OLS   Adj. R-squared:                  0.673
Method:                 Least Squares   F-statistic:                     1524.
Date:                Thu, 23 Jul 2020   Prob (F-statistic):               0.00
Time:                        17:20:19   Log-Likelihood:                -31170.
No. Observations:                2222   AIC:                         6.235e+04
Df Residuals:                    2218   BIC:                         6.237e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept    -5.196e+04   8244.042     -6.303      0.000   -6.81e+04   -3.58e+04
bolingerdiff     0.1090      0.005     23.901      0.000       0.100       0.118
dist             0.1710      0.011     15.398      0.000       0.149       0.193
volume           9.7457      0.529     18.417      0.000       8.708      10.783
==============================================================================
Omnibus:                     2316.052   Durbin-Watson:                   1.085
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           423811.988
Skew:                           4.682   Prob(JB):                         0.00
Kurtosis:                      70.007   Cond. No.                     3.58e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.58e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
for coin in coin_list:
    if df_coin_dict[coin].iloc[-1]['bol'] < 0.1:
        print(coin)